In [6]:
import json

# تحميل البيانات من ملف JSON
with open(r"D:\Desktop\Programming\LangChain - learning\ice_breaker\my_projects\agentsqlprojectbot\token_logs\chat_5283091780_costs.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# عدد الطلبات الحالية حسب الملف
current_requests = 999  # حسب ما ذكرت
future_requests = 10**6

total_cost_all_models = 0
estimated_total_cost_all_models = 0

for model in data.get("models", []):
    # تحديث سعر gemini-2.5-flash
    if model["name"] == "gemini-2.5-flash":
        model["pricing"]["output_per_1m"] = 2.5
        model["pricing"]["input_per_1m"] = 0.3

    # التكلفة الحالية
    input_cost = (model["tokens"]["input"] / 1_000_000) * model["pricing"]["input_per_1m"]
    output_cost = (model["tokens"]["output"] / 1_000_000) * model["pricing"]["output_per_1m"]
    total_cost = input_cost + output_cost
    total_cost_all_models += total_cost

    # التكلفة المستقبلية (10,000 طلب)
    scale_factor = future_requests / current_requests
    estimated_total_cost = total_cost * scale_factor
    estimated_total_cost_all_models += estimated_total_cost

    print(f"Model: {model['name']}")
    print(f"  Current Total Cost: ${total_cost:.6f}")
    print(f"  Estimated Total Cost for {future_requests} requests: ${estimated_total_cost:.2f}")
    print("-" * 40)

print(f"Total Current Cost (all models): ${total_cost_all_models:.6f}")
print(f"Estimated Total Cost for {future_requests} requests (all models): ${estimated_total_cost_all_models:.2f}")


Model: gemini-2.5-flash
  Current Total Cost: $0.074689
  Estimated Total Cost for 1000000 requests: $74.76
----------------------------------------
Model: gemini-2.0-flash
  Current Total Cost: $0.003852
  Estimated Total Cost for 1000000 requests: $3.86
----------------------------------------
Total Current Cost (all models): $0.078541
Estimated Total Cost for 1000000 requests (all models): $78.62


In [1]:
from sqlalchemy import create_engine, MetaData, inspect

# رابط الاتصال
connection_string = "mssql+pyodbc://@B515R\\SQLEXPRESS/manager?driver=ODBC+Driver+17+for+SQL+Server"

# إنشاء محرك SQLAlchemy
engine = create_engine(connection_string)

# إنشاء كائن Metadata
metadata = MetaData()

# عمل Reflect لجميع الجداول
metadata.reflect(bind=engine)

# استخدام Inspector للحصول على معلومات أكثر تفصيلًا
inspector = inspect(engine)

print("=== Tables in database ===")
for table_name in inspector.get_table_names():
    print(f"\nTable: {table_name}")
    
    # أعمدة الجدول
    columns = inspector.get_columns(table_name)
    for col in columns:
        print(f"  {col['name']} ({col['type']}) Nullable: {col['nullable']} Default: {col['default']}")

    # المفاتيح الأساسية
    pk = inspector.get_pk_constraint(table_name)
    print(f"  Primary Key: {pk.get('constrained_columns')}")

    # المفاتيح الأجنبية
    fks = inspector.get_foreign_keys(table_name)
    for fk in fks:
        print(f"  Foreign Key: {fk.get('constrained_columns')} -> {fk.get('referred_table')}({fk.get('referred_columns')})")


=== Tables in database ===

Table: dashboard_users
  id (INTEGER) Nullable: False Default: None
  org_id (NVARCHAR(100) COLLATE "SQL_Latin1_General_CP1_CI_AS") Nullable: False Default: None
  user_id (BIGINT) Nullable: False Default: None
  username (NVARCHAR(100) COLLATE "SQL_Latin1_General_CP1_CI_AS") Nullable: False Default: None
  password_hash (NVARCHAR(500) COLLATE "SQL_Latin1_General_CP1_CI_AS") Nullable: False Default: None
  role (NVARCHAR(50) COLLATE "SQL_Latin1_General_CP1_CI_AS") Nullable: False Default: None
  is_active (BIT) Nullable: True Default: ((1))
  created_at (DATETIME) Nullable: True Default: (getdate())
  Primary Key: ['id']
  Foreign Key: ['org_id'] -> organizations(['org_id'])

Table: database_connections
  connection_id (NVARCHAR(100) COLLATE "SQL_Latin1_General_CP1_CI_AS") Nullable: False Default: None
  name (NVARCHAR(255) COLLATE "SQL_Latin1_General_CP1_CI_AS") Nullable: False Default: None
  connection_string (NVARCHAR COLLATE "SQL_Latin1_General_CP1_CI_A